In [ ]:
"""
Created on Mon Jan 13 16:00:47 2020
Group 7
@authors : G.H.
"""

In [15]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread
import numpy as np
import pandas as pd
import math
import operator
import os
from PIL import Image
from tqdm import tqdm_notebook as tqdm
import random
import pickle

import shutil
from shutil import copyfile

# keras : librairie de deep learning
import keras
from keras.models import Sequential, Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, InputLayer, ReLU, AveragePooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

# Lecure avec keras

In [2]:
# Reading of csv file SEATGURU annontate
csv_file = pd.read_csv('../g7_SEATGURU_annotate.csv', sep=';')
Airbus_file = csv_file.loc[csv_file['aircraft_manufacturer']  == 'Airbus',:]
Airbus_file = Airbus_file.loc[Airbus_file['view'] == 'Int']
Boeing_file = csv_file.loc[csv_file['aircraft_manufacturer']  == 'Boeing',:]
Boeing_file = Boeing_file.loc[Boeing_file['view'] == 'Int']
# List name pictures
list_img_Airbus = list(Airbus_file['name'])
list_img_Boeing = list(Boeing_file['name'])

In [3]:
path = './../All Data/ANALYSE IMAGE/IMG SEATGURU/'
new_path_train = './../G7_SEATGURU_cat/Int/Split_data_man/data_train'
new_path_val = './../G7_SEATGURU_cat/Int/Split_data_man/data_val'
new_paths = [new_path_train, new_path_val]

In [7]:
def split_train_test_Seatguru_cat(new_paths, path, list_planes = [list_img_Airbus, list_img_Boeing], 
                                  split_limit = .7, s = 8):
  

    for plane in list_planes:
        
        if plane == list_img_Airbus:
            classe = 'Airbus'
        if plane == list_img_Boeing:
            classe = 'Boeing'
        
        shutil.rmtree(new_paths[0] + '/' + classe, ignore_errors = True)
        os.makedirs(new_paths[0] + '/' + classe, exist_ok = True)
        shutil.rmtree(new_paths[1] + '/' + classe, ignore_errors = True)
        os.makedirs(new_paths[1]+ '/' + classe, exist_ok = True)
        
        random.seed(a = s)
        random.shuffle(plane)

        for pict in plane[:int(split_limit*len(plane))]:
            copyfile(path + '/' + pict, new_paths[0] + '/' + classe + '/' +  pict)
        for pict in plane[int(split_limit*len(plane)):]:
            copyfile(path + '/' + pict, new_paths[1] + '/' + classe + '/' +  pict)
        

In [8]:
# Creation of train and test samples
split_train_test_Seatguru_cat(new_paths, path)

In [9]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies


train_generator = train_datagen.flow_from_directory(new_path_train,
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

test_datagen = ImageDataGenerator(preprocessing_function = preprocess_input)

test_generator = test_datagen.flow_from_directory(new_path_val,
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 1123 images belonging to 2 classes.
Found 482 images belonging to 2 classes.


# Transfer learning

In [11]:
# create the base pre-trained model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# add a global spatial average pooling layer
x = base_model.output


x = Conv2D(256, kernel_size=(3, 3))(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = Conv2D(256, kernel_size=(3, 3))(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = AveragePooling2D(pool_size=(2, 2))(x)

# vector
x = Flatten()(x)

# let's add a fully-connected layer
x = Dense(1024)(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = Dense(512)(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = Dense(256)(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)


# last layer which predict the class
predictions = Dense(2, activation = 'softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [12]:
step_size_train=train_generator.n//train_generator.batch_size

# Define learning rate reducer
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', 
                              patience=2, 
                              verbose=1, 
                              factor=0.5, 
                              min_lr=0.00001)

# How to include it in your training
model.fit_generator(generator=train_generator,
                    steps_per_epoch=step_size_train,
                    epochs=10,
                    validation_data=test_generator,
                    callbacks=[reduce_lr])  # here

Epoch 1/10
35/35 [==============================] - 629s 18s/step - loss: 0.9073 - accuracy: 0.5307 - val_loss: 1.4630 - val_accuracy: 0.4751
Epoch 2/10
35/35 [==============================] - 1607s 46s/step - loss: 0.4780 - accuracy: 0.7764 - val_loss: 1.2494 - val_accuracy: 0.5290
Epoch 3/10
35/35 [==============================] - 468s 13s/step - loss: 0.2682 - accuracy: 0.9038 - val_loss: 0.1962 - val_accuracy: 0.5809
Epoch 4/10
35/35 [==============================] - 475s 14s/step - loss: 0.1774 - accuracy: 0.9413 - val_loss: 0.9547 - val_accuracy: 0.5871
Epoch 5/10
35/35 [==============================] - 467s 13s/step - loss: 0.2661 - accuracy: 0.9239 - val_loss: 1.0265 - val_accuracy: 0.5871
Epoch 6/10
35/35 [==============================] - 472s 13s/step - loss: 0.1543 - accuracy: 0.9366 - val_loss: 3.6907 - val_accuracy: 0.6349
Epoch 7/10
35/35 [==============================] - 459s 13s/step - loss: 0.2691 - accuracy: 0.9190 - val_loss: 0.1033 - val_accuracy: 0.5851
Epoch

In [16]:
def save_model_classes(path_mod, mod_name, train_generator, model):
    
    shutil.rmtree(path_mod + mod_name, ignore_errors = True)
    os.makedirs(path_mod + mod_name)
    label_map = (train_generator.class_indices)
    with open(path_mod + mod_name + '/' + 'model_' + mod_name + '.pkl', "wb") as f:
        pickle.dump(label_map, f)
    model.save(path_mod + mod_name + '/' + 'model_'+ mod_name + '.h5')

In [17]:
save_model_classes("./../Models/", 'int_man', train_generator, model)